Dans ce tutoriel nous verrons comment créer un réseau de neurones récurent, pour cela nous utiliserons le problème du MNIST afin de voir comment créer et entrainer une couche simple d'un réseau récurent, puis nous aborderont le problème de la prédiction de texte afin d'apprendre à créer des réseaux récurents multi-couche.

# Problème du MNIST et réseau simple couche

Afin de créer une couche d'un réseau récurent il nous faut passer par la fonction BasicLSTMCell de TensorFlow, vous pourrez trouver toutes les information necessaire à l'utilisation de cette fonction sur [BasicLSTMCell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell)

Nous aurons aussi besoin de la fonction staticRNN de TensorFlow pour ce tutoriel, vous trouvez le necessaire sur [staticRNN](https://www.tensorflow.org/api_docs/python/tf/nn/static_rnn)

Commencons par importer les librairies utiles:

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# importer ici les librairies necessaires au fonction BasicLSTMCell et staticRNN

On va aussi télecharger et enregistrer la base de donnée du MNIST:

In [5]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


On va maintenant définir les variables globales pour notre réseau ainsi que créer les placeholders pour l'entrée et la sortie de notre réseau:

In [6]:
hm_epochs = 10
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128


x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

Nous allons pouvoir créer notre procédure qui créera le réseau récurent, certaines parties seront à completer par vous même:

In [ ]:
def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size,n_classes])), # ici nous créons les poids et les biais pour notre réseau
             'biases':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size]) # nous devons redimensionner l'entrée afin de la faire correspondre pour notre réseau
    x = tf.split(x, n_chunks, 0)

    lstm_cell = # Créer la cellule pour le réseau récurent grâce a la fonction BasicLSTMCell, passer en paramètre la rnn_size et mettre le paramètre state_is_tuple à true
    outputs, states = # créer le réseau a l'aide de la fonction static_rnn en passant en paramètre la cellule créée précedemment ainsi que l'entrée x et indiquer dtype=tf.float32

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

Nous allons maintenant créer la fonction qui sevira a l'entrainement de notre réseau récurent.

In [ ]:
def train_neural_network(x):
    prediction = recurrent_neural_network(x) # on créer le réseau
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y, name = None) )
    optimizer = tf.train.AdamOptimizer().minimize(cost) # on définit les fonction de cout et d'optimisation
    
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size)) # On redimensionne les données avant de les mettre dans notre réseau afin que celle-ci soit accepter par ce dernier

                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y}) # on fait tourner le réseau avec les données redimensionnées
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels})) # on affiche les résultats obtenus à la fin de l'entrainement

Il nous suffit à présent d'executer la fonction d'entrainement créée précedement.

In [ ]:
train_neural_network(x)

Nous avons donc maintenant vu comment créer un réseau récurent ne comportant qu'une seule couche de neurone, dans la suite nous verrons comment créer un réseau multi-couche sur le problème de la prédiction de texte.

# Réseau multi-couche et problème de prédiction de texte

Afin de créer un réseau multi-couche nous aurons besoin de la fonction [MultiRNNCell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell) de TensorFlow, aussi afin de résoudre le problème de la prédiction de texte nous aurons besoin d'un petit texte pour l'entrainement du réseau, je vous propose le texte ci dessus que nous enregistrerons dans un fichier texte nommé belling_the_cat.txt :

long ago , the mice had a general council to consider what measures they could take to outwit their common enemy , the cat . some said this , and some said that but at last a young mouse got up and said he had a proposal to make , which he thought would meet the case . you will all agree , said he , that our chief danger consists in the sly and treacherous manner in which the enemy approaches us . now , if we could receive some signal of her approach , we could easily escape from her . i venture , therefore , to propose that a small bell be procured , and attached by a ribbon round the neck of the cat . by this means we should always know when she was about , and could easily retire while she was in the neighbourhood . this proposal met with general applause , until an old mouse got up and said that is all very well , but who is to bell the cat ? the mice looked at one another and nobody spoke . then the old mouse said it is easy to propose impossible remedies .

Commencons par importer les librairies utiles:

In [7]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import random
import collections
import time
# importer ici la librairie pour les fonctions BasicLSTMCell, MultiRNNCell et static_rnn

Nous allons maintenant définir quelque procédure utiles à notre programme:

In [11]:
# Pour convertir le temp du format sec* à hh:mm:ss
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

# Pour lire le fichier texte et en récuperer les mots
def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

# Pour traiter les mots récuperés
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

Nous allons maintenant pouvoir traiter le fichier belling_the_cat.txt afin d'en créer une base de données pour l'entrainement.

In [13]:
# Target log path
logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = 'belling_the_cat.txt'

training_data = read_data(training_file)
print("Loaded training data...")

dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

Loaded training data...


Définissons les différents paramètres du réseau ainsi que les placeholders:

In [14]:
# Parameters
learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 256

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

Pour la suite nous allons créer une procedure qui créera un réseau récurent multi-couche, certaines parties de ce code seront à complèter.

In [ ]:
def RNN(x, weights, biases):

    # RNN output node weights and biases
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([vocab_size]))
    }

    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = # utiliser la fonction MultiRNNCell afin de créer un réseau composé de 2 BasicLSTMCell de taille n_hidden

    # generate prediction
    outputs, states = # utiliser ici la fonction static_rnn de la même manière que lors du dernier exercice

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

Nous allons maintenant créer le réseau grâce à la procedure RNN puis nous l'entrainerons:

In [ ]:
start_time = time.time()

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")